# Лабораторная работа

Баллы за секции:
- *Задание 1* - 10 баллов
- *Задание 2* - 20 баллов
- *Задание 3* - 30 баллов
- **Итого** - 60 баллов

Оценка за лабу $ = \frac{score}{6}$

## Внимание! Attention! Achtung!

**Данная лабораторная работа является блокирующей**, т.е. если лабораторная работа не выполнена, то оценка за курс - неуд.

Лабораторная работа считается выполненной, если **за каждое из заданий получено не менее 2 баллов**.

## Формат сдачи лабораторной работы

Данная лабораторная работа сдается так же, как и проект - заполняете данный ноутбук, отправляете его через GitHub с указанием проверяющего.

**Ассистенты не будут запускать ячейки кода, а смотреть на выведенный результат и код. Если ячейка кода не запущена - балл не ставится, даже при правильно написанном коде.**

***(Комментарии что курс по Python легче чем по C++ учтены)***

>**Примечание**
>
>Данные задания вдохновлены предметом "Введение в анализ данных" 2 курса, по итогу которого идёт отбор на DS-поток. Если вы видите себя в смежных областях, советуем выполнить данную лабораторку по максимуму.

In [2]:
# Всё необходимое для выполнения данной лабораторной работы!
import requests
from time import sleep

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Задание 1. Парсинг - 10 баллов

В этом задании Вам предстоит работать с API сайта <a target="_blank" href="hh.ru">hh.ru</a>. Документация доступна по <a target="_blank" href="https://github.com/hhru/api/blob/master/docs/vacancies.md#search">ссылке</a>.

<b><font color="red">Внимание!</font> При работе с API не забывайте делать паузы между запросами, иначе вас забанят. Просрочка дедлайна по этой причине не принимается!</b>

**Задача:**
1. выбрать 5 интересующих вас профессий (не смежных, т.е. Аналатик и Data Engineer не считается корректным)
2. затем получить доступные по данным профессиям вакансии в Москве и выделить их физические координаты
3. нанести полученные координаты на карту местоположения компаний

*Пример запроса для профессии:*

In [3]:
URL = 'https://api.hh.ru/vacancies'

params = {
    'text': "Физик-ядерщик",
    'area': 1,
    'page': 0,
    'per_page': 10
}

data = requests.get(URL, params).json()

Если Ваш запрос прошел успешно, то вы получите следующие ключи (значения данных ключей вы найдете по ссылке на документацию):

In [4]:
data.keys()

dict_keys(['items', 'found', 'pages', 'per_page', 'page', 'clusters', 'arguments', 'alternate_url'])

>**Примечание**
>
>В `data['pages']` будет хранится количество найденных страниц вакансий - для корректного сбора данных Вам нужно пробежать по **КАЖДОЙ** странице и сделать соотвествующие запросы.

>**Совет**
>
>Посмотрите что хранится в `items`!

(1 балл) Выберете для нас 5 наиболее интересных специальностей и получите по ним имеющиеся вакансии:

In [5]:
jobs = ["Каменщик", "Сварщик", "Уборщик", "Пивовар", "С++ разработчик"]

def get_job(job):
    req = requests.get(URL, {
                              'text': job,
                              'area': 1,
                              'page': 0,
                              'per_page': 100
                              }
                        ).json()
    sleep(1)
    cnt = req['pages']
    ret = list()
    for i in range(0, cnt):
        ret += requests.get(URL, {
                              'text': job,
                              'area': 1,
                              'page': i,
                              'per_page': 100
                              }
                        ).json()['items']
        
        sleep(1)
    return ret

vacs = list()
for j in jobs:
    vacs += get_job(j)

In [17]:
vacs

[{'id': '80388525',
  'premium': False,
  'name': 'Каменщик',
  'department': None,
  'has_test': False,
  'response_letter_required': False,
  'area': {'id': '1', 'name': 'Москва', 'url': 'https://api.hh.ru/areas/1'},
  'salary': {'from': 100000, 'to': None, 'currency': 'RUR', 'gross': False},
  'type': {'id': 'open', 'name': 'Открытая'},
  'address': {'city': 'Москва',
   'street': 'Николоямская улица',
   'building': '40/22с4',
   'lat': 55.746785,
   'lng': 37.659308,
   'description': None,
   'raw': 'Москва, Николоямская улица, 40/22с4',
   'metro': {'station_name': 'Таганская',
    'line_name': 'Кольцевая',
    'station_id': '5.76',
    'line_id': '5',
    'lat': 55.742396,
    'lng': 37.653334},
   'metro_stations': [{'station_name': 'Таганская',
     'line_name': 'Кольцевая',
     'station_id': '5.76',
     'line_id': '5',
     'lat': 55.742396,
     'lng': 37.653334}],
   'id': '2891372'},
  'response_url': None,
  'sort_point_distance': None,
  'published_at': '2023-05-14T00

(1 балл) Посчитайте сколько всего найдено вакансий:

In [6]:
len(vacs)

3488

(1 балл) Создайте структуру данных, которая будет хранить в себе имя компании и ее координаты (широта и долгота):

In [10]:
class Company:
    def __init__(self, name, x, y):
        self.name = name
        self.x = x
        self.y = y

(1 балл) Создайте экземпляр этой структуры данных для каждой компании, которая выставила вакансию:

In [16]:
comps = dict()

for vac in vacs:
        name = vac['employer']['name']
        if name not in comps.keys():
            addr = vac['address']
            x__ = None
            y__ = None
            if addr and addr['lat']:
                x__ = addr['lat']
                y__ = addr['lng']
            comps[name] = Company(name, x__, y__)
            
comps

{'АльпКор': <__main__.Company at 0x1b26683be20>,
 'Стройкомфорт24': <__main__.Company at 0x1b269175f30>,
 'ИБТ': <__main__.Company at 0x1b269174be0>,
 'БМК ТРАНСМОСТ': <__main__.Company at 0x1b269175360>,
 'Ростпроектмонтаж': <__main__.Company at 0x1b269176b60>,
 'Моспромстрой': <__main__.Company at 0x1b2691751b0>,
 'Лазер Про Клиник': <__main__.Company at 0x1b269175f00>,
 'Раминжиниринг': <__main__.Company at 0x1b269175180>,
 'Управление Развития Строительных Технологий (АО УРСТ)': <__main__.Company at 0x1b2691760b0>,
 'ПИК-специализированный застройщик': <__main__.Company at 0x1b269176380>,
 'К.С.Бюро': <__main__.Company at 0x1b269176080>,
 'АРБА Дистрибьюшн': <__main__.Company at 0x1b269176440>,
 'ГК Альянс': <__main__.Company at 0x1b2691765c0>,
 'ГК Олимпроект': <__main__.Company at 0x1b269176680>,
 'Строительная компания ФЛАН - М': <__main__.Company at 0x1b269176770>,
 'АЛЬФАВЕСТСТРОЙ': <__main__.Company at 0x1b269176620>,
 'Яндекс': <__main__.Company at 0x1b2691764a0>,
 'ADAMAS':

(1 балл) Найдите количество вакансий, у которых заданы координаты (значения данных может быть пустым):

In [15]:
non_empty = len([x for x in comps.values() if x.x != None])

non_empty

1248

(1 балл) Отберите вакансии, которые расположены в пределах МКАД (можно грубо):

In [ ]:
# your code here

(1 балл) Постройте график в координатах "широта-долгота" для отображения вакансий внутри МКАД (не забывайте прописать прозрачность, чтобы увидеть скопления):

In [ ]:
# your code here

(1 балл) Нанесите все точки вакансий на карту (для этого вам поможет данная <a target="_blank" href="https://plotly.com/python/mapbox-layers/">документации</a>, не забудьте про прозрачность и центрирование по Москве): 

In [ ]:
# your code here

(2 балла) **Сделайте выводы о проделанной работе:**

>*Подсказки для маленьких: Что вы можете сказать про районы Москвы? Что вы можете сказать про зарплаты? Что вы можете сказать про спрос?*

Я делаю по фасту на минимальный балл, так что могу вывести, что вакансий достаточно много

## Задание 2. Анализ данных - 20 баллов

Вам предстоит проанализировать датасет, содержащий информацию об бриллианта, их стоимости и остальных характеристиках.



Поясним значения хранящиеся в колонках:
*   `cut` - огранка бриллианта (относится к одной из примерно 10 наиболее распространенных)
*   `color` - цвет бриллианта, прозрачные бриллианты имеют градацию `D-Z`, "более высокие" буквы - более желтоватые, но часто имеют лучшие значения, поскольку цвет трудно определить один раз в кольце
*   `clarity` - чистота бриллианта, наличие включений (внутренних дефектов), чем меньше и мельче, тем лучше
*   `carat_weight` - масса бриллианта (слабо связана с размером бриллианта, т.к. огранка и качество огранки, как правило, играют большую роль)
*   `cut_quality` - качество ограники по системе оценки GIA Cut
*   `lab` - лаборатория оценивания
*   `meas_length` - длина бриллианта
*   `meas_width` - ширина бриллианта
*   `meas_depth` - глубина бриллианта
*   `total_sales_price` - цена бриллианта в долларах.
*    **и прочие колонки**

Скачаем данные:

In [19]:
!pip install gdown

In [20]:
!gdown 1uIWeZFz3ElB5p1brBVEwJBHTbjXNPdfU

Downloading...
From: https://drive.google.com/uc?id=1uIWeZFz3ElB5p1brBVEwJBHTbjXNPdfU
To: C:\Users\Dubvas\Downloads\diamonds.csv

  0%|          | 0.00/35.4M [00:00<?, ?B/s]
  1%|1         | 524k/35.4M [00:00<00:08, 4.36MB/s]
  3%|2         | 1.05M/35.4M [00:00<00:11, 2.96MB/s]
  4%|4         | 1.57M/35.4M [00:00<00:12, 2.70MB/s]
  6%|5         | 2.10M/35.4M [00:00<00:12, 2.59MB/s]
  7%|7         | 2.62M/35.4M [00:01<00:13, 2.45MB/s]
  9%|8         | 3.15M/35.4M [00:01<00:12, 2.60MB/s]
 10%|#         | 3.67M/35.4M [00:01<00:12, 2.53MB/s]
 12%|#1        | 4.19M/35.4M [00:01<00:13, 2.32MB/s]
 13%|#3        | 4.72M/35.4M [00:01<00:13, 2.31MB/s]
 15%|#4        | 5.24M/35.4M [00:02<00:12, 2.49MB/s]
 16%|#6        | 5.77M/35.4M [00:02<00:12, 2.40MB/s]
 18%|#7        | 6.29M/35.4M [00:02<00:11, 2.44MB/s]
 19%|#9        | 6.82M/35.4M [00:02<00:12, 2.23MB/s]
 21%|##        | 7.34M/35.4M [00:02<00:11, 2.50MB/s]
 22%|##2       | 7.86M/35.4M [00:03<00:11, 2.50MB/s]
 24%|##3       | 8.39M/35.4M [00

Создадим DataFrame:

In [21]:
diamonds = pd.read_csv("diamonds.csv")

(0.5 балла) Выведете первые 5 строк датасета:

In [22]:
diamonds[0:5]

,Unnamed: 0,cut,color,clarity,carat_weight,cut_quality,lab,symmetry,polish,eye_clean,...,meas_depth,girdle_min,girdle_max,fluor_color,fluor_intensity,fancy_color_dominant_color,fancy_color_secondary_color,fancy_color_overtone,fancy_color_intensity,total_sales_price
0,0,Round,E,VVS2,0.09,Excellent,IGI,Very Good,Very Good,unknown,...,1.79,M,M,unknown,None,unknown,unknown,unknown,unknown,200
1,1,Round,E,VVS2,0.09,Very Good,IGI,Very Good,Very Good,unknown,...,1.78,STK,STK,unknown,None,unknown,unknown,unknown,unknown,200
2,2,Round,E,VVS2,0.09,Excellent,IGI,Very Good,Very Good,unknown,...,1.77,TN,M,unknown,None,unknown,unknown,unknown,unknown,200
3,3,Round,E,VVS2,0.09,Excellent,IGI,Very Good,Very Good,unknown,...,1.78,M,STK,unknown,None,unknown,unknown,unknown,unknown,200
4,4,Round,E,VVS2,0.09,Very Good,IGI,Very Good,Excellent,unknown,...,1.82,STK,STK,unknown,None,unknown,unknown,unknown,unknown,200


(0.5 балла) Найдите количество бриллиантов всего в данном датасете:

In [31]:
len(diamonds)

219703

(0.5 балла) Выведите максимальную и минимальную цену бриллианта в формате `номер - цена`:

In [24]:
prices = diamonds['total_sales_price']
imx = prices.idxmax()
imn = prices.idxmin()
print(imn, '-', prices[imn])
print(imx, '-', prices[imx])

0 - 200
219702 - 1449881


(1 балл) Посчитайте количество пропусков (пропуском считается значение `unknown` или `None`):

In [43]:
((diamonds=='unknown') | diamonds.isnull() | (diamonds=='None')).sum().sum()

1891168

(1 балл) Найдите в каких столбцах присутствуют пропуски:

In [46]:
cols = [c for c in diamonds.columns if diamonds[c].isin(['unknown', 'None']).sum() > 0]
cols

['color',
 'cut_quality',
 'eye_clean',
 'culet_size',
 'culet_condition',
 'girdle_min',
 'girdle_max',
 'fluor_color',
 'fluor_intensity',
 'fancy_color_dominant_color',
 'fancy_color_secondary_color',
 'fancy_color_overtone',
 'fancy_color_intensity']

В нашем датасете присутствуют колонки, которые начинаются с `fancy_color_` или `fluor_`. Большинство значений в этих колонках - `unknown` или `None`, поэтому мы считаем их малоинформативными. 

Ваша задача:

1. (0.5 балла) найти количество таких колонок, которые начинаются с `fancy_color_` или `fluor_`:

In [ ]:
# your code here

2. (0.5 балла) найти количество информативных (не `None` или `unknown`) значений в этих колонках: 

In [ ]:
# your code here

3. (0.5 балла) удалить данные колонки из датасета, сохранив изменения (в дальнейшем работаем измененным датасетом):

In [ ]:
# your code here

(1 балл) Проделайте тоже самое для колонок, начинающихся с `culet_`:

In [ ]:
# your code here

(1 балл) Посчитайте сколько бы стоил (в тысячах долларов) сейф, в котором хранились бы все бриллианты из датасета:

In [ ]:
# your code here

Немного необоснованных манипуляций.

(1 балл) Выберите из датасета строки с четными индексами и выведете значения (не меняя нумерацию индексов):

In [ ]:
# your code here

(0.5 балла) А теперь с индексами, кратными 6:

In [ ]:
# your code here

(1 балл) Выберете из датасета только категорильные признаки и выведете датасет:

In [ ]:
# your code here

>**Примечание**
>
>Данный датасет можно использовать для задания 3 (с использованием `sklearn`)

(0.5 балла) Выберете столбец с данными о цене бриллиантов и домножите его на тот столбец, только транспонированный, и выведите полученную матрицу:

In [ ]:
# your code here

### Построение графиков

**Требования к графику:**

1.   График должен быть подписан
2.   Оси должны быть подписаны
3.   Должна быть легенда графика
4.   График должен быть информативным (желательно без сильных выбросов)

**Все надписи на русском языке!**

>**Примечание**
>
>График необходимо строить для очищенных данных - в них не должно быть пропусков по осям, по которым строите график. При этом **не перезатирайте** имеющийся датафрейм!

(2 балла) Постройте график `цвет бриллианта - стоимость`:

In [ ]:
# your code here

(2 балла) Постройте график `лабараторная оценка - кол-во бриллиантов`:

In [ ]:
# your code here

(2 балла) Постройте график `вес бриллианта - стоимость`:

In [ ]:
# your code here

(2 балла) Постройте график `система оценки GIA Cut - кол-во бриллиантов`:

In [ ]:
# your code here

(2 балла) **Сделайте выводы по построенным графикам:**

>*Подсказка для маленьких: Что вы можете сказать о графиках полученных выше для данного датасета?*

Ваши выводы здесь :)

## Задание 3. Модель - 30 баллов

В данном задании вам предстоит построить классифицирующую модель размер алмаза и его стоимости, а затем оценить полученную модель.

>**Примечание**
>
>Данная модель не претендует на самый верный индикатор стоимости алмаза, ведь мы не учитываем ничего больше чем стоимость алмаза. Исключительно для образовательных целей :)

Для построения модели нам понадобится 4 столбца, а именно: 
- `meas_length`
- `meas_width`
- `meas_depth`
- `total_sales_price`

От остальных столбцов можно избавиться.

(1 балл) Создайте новый датасет и выведите его первые 10 строк: 

In [51]:
ndiams = diamonds.drop(['Unnamed: 0', 'cut', 'color', 'clarity', 'carat_weight', 'cut_quality', 'lab', 'symmetry', 'polish', 'eye_clean', 'culet_size', 'culet_condition', 'depth_percent','table_percent', 'girdle_min', 'girdle_max', 'fluor_color', 'fluor_intensity', 'fancy_color_dominant_color','fancy_color_secondary_color', 'fancy_color_overtone', 'fancy_color_intensity'], axis=1)
ndiams[0:10]

,meas_length,meas_width,meas_depth,total_sales_price
0,2.85,2.87,1.79,200
1,2.84,2.89,1.78,200
2,2.88,2.90,1.77,200
3,2.86,2.88,1.78,200
4,2.79,2.83,1.82,200
5,2.95,2.99,1.81,200
6,2.85,2.88,1.84,200
7,2.86,2.89,1.78,200
8,2.89,2.92,1.85,200
9,2.83,2.87,1.80,200


(1 балл) Перемешайте строки в датафрейме случайным образом: 

In [52]:
ndiams = ndiams.sample(frac=1).reset_index(drop=True)
ndiams

,meas_length,meas_width,meas_depth,total_sales_price
0,4.68,4.73,2.95,1960
1,3.82,3.84,2.34,574
2,4.81,5.62,3.09,2736
3,3.95,3.97,2.46,818
4,5.12,5.15,3.22,2900
...,...,...,...,...
219698,6.39,6.41,4.01,4068
219699,4.30,4.32,2.72,838
219700,6.97,4.92,3.11,3360
219701,5.04,5.07,3.17,3090


(1 балл) Сформируем новую колонку - для каждого бриллианта найдем сумму Минковского (p=3) по 3 измерениям бриллианта:

In [55]:
def P3(a, b, c):
    m1 = abs(a - b)
    m2 = abs(a - c)
    m3 = abs(b - c)
    return (m1 ** 3 + m2 ** 3 + m3 ** 3) ** (1/3)

ndiams.insert(3, 'mink_p', P3(ndiams['meas_length'], ndiams['meas_width'], ndiams['meas_depth']))
ndiams

,meas_length,meas_width,meas_depth,mink_p,total_sales_price
0,4.68,4.73,2.95,2.211619,1960
1,3.82,3.84,2.34,1.877368,574
2,4.81,5.62,3.09,2.794127,2736
3,3.95,3.97,2.46,1.889966,818
4,5.12,5.15,3.22,2.412898,2900
...,...,...,...,...,...
219698,6.39,6.41,4.01,3.011264,4068
219699,4.30,4.32,2.72,2.003354,838
219700,6.97,4.92,3.11,4.161271,3360
219701,5.04,5.07,3.17,2.375103,3090


(1 балл) Сформируем новый датасет из двух колонок - сумма Минковского и `total_sales_price`: 

In [57]:
newestdiams = ndiams.drop(['meas_length', 'meas_width', 'meas_depth'], axis=1)
newestdiams

,mink_p,total_sales_price
0,2.211619,1960
1,1.877368,574
2,2.794127,2736
3,1.889966,818
4,2.412898,2900
...,...,...
219698,3.011264,4068
219699,2.003354,838
219700,4.161271,3360
219701,2.375103,3090


(1 балл) Разделите вашу выборку на две части в соотношении 8:2 - одна выборка будет для тренировка, а другая для тестов вашей модели:

In [58]:
diamsz = len(newestdiams)
train = newestdiams[:diamsz * 8 // 10].reset_index(drop=True)
test = newestdiams[diamsz * 8 // 10:].reset_index(drop=True)
train

,mink_p,total_sales_price
0,2.211619,1960
1,1.877368,574
2,2.794127,2736
3,1.889966,818
4,2.412898,2900
...,...,...
175757,2.116743,1088
175758,4.284069,18440
175759,3.352474,16191
175760,2.923237,6632


In [59]:
test

,mink_p,total_sales_price
0,1.110076,1870
1,2.160932,888
2,3.238046,7938
3,1.360381,10800
4,7.629668,18626
...,...,...
43936,3.011264,4068
43937,2.003354,838
43938,4.161271,3360
43939,2.375103,3090


(2 балла) Постройте график зависимости стоимости от вашего нового параметра размера для тренинг-выборки:

In [ ]:
# your code here

(2 балла) Нанесите на тот же график тестовую выборку, используя другой цвет (не забудьте про параметр прозрачности):

In [ ]:
# your code here

(3 балла) Найдите для каждой бриллианта из тестовой выборки его "ближайшего соседа" из тренинг-выборки. Для каждого бриллианта из тестовой выборки найдите расстояния до соответствующего ближайшего соседа:

In [ ]:
# your code here

(3 балла) Найдите насколько сильно (в процентах) отличается цена бриллиантов из тестовой выборки от их соответствующих ближайших соседей:

In [ ]:
# your code here

**Данная модель является выдумкой автора. Не важен коэффициент ошибки, важны ваши шаги для получения коэффициента.**

### Введение в машинное обучение

Теперь давайте познакомимся с таким сильным инструментом как `sklearn`.

Вам необходимо:
1. выбрать 2 категориальных признака
2. на их основе обучить решающее дерево
3. оценить среднеквадратичную ошибку цены бриллианта
4. сделать пп. 1-3 и выбрать ту пару, которая дает лучший результат
5. сделать выводы

(3 балла) Мы работаем с категориальными признаками, для них не понятно что есть "больше" и "меньше". Поэтому нам нужно предварительно закодировать значения наших признаков, с чем нам поможет [OneHotEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):

In [ ]:
# your code here

(3 балла) Разбейте датасет на обучающую и тестовую выбору. Для этого вам поможет [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html):

In [ ]:
# your code here

(3 балла) Затем создайте и обучите вашу модель - [DecisionTreeRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html):

In [ ]:
# your code here

(3 балла) Оцените ошибку вашей модели при помощи [mean_squared_error](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html):

In [ ]:
# your code here

(3 балла) **Сделайте выводы какие категориальные признаки лучше:**

*Подсказки для маленьких: Что вы можете сказать о зависимости по вашей выборке признаков? Какой вы можете сделать вывод об готовых инструментах для работы и написанных Вами?*

Ваши выводы здесь :)